In [51]:
import pandas as pd

# Read in CSV File
# season_box = pd.read_csv('data/box_scores/season_box_scores_20182019.csv', converters={'GAME_ID': str, 'TEAM_ABBREVIATION': str}, index_col=False)
# season_games = pd.read_csv('data/games/season_games_20182019.csv', converters={'Game_ID': str, 'TEAM_ABBREVIATION': str}, index_col=False)\

# season_box = pd.read_csv('data/box_scores/season_box_scores_20192020.csv',
#                          converters={'GAME_ID': str, 'TEAM_ABBREVIATION': str}, index_col=False)
# season_games = pd.read_csv('data/games/season_games_20192020.csv', converters={'Game_ID': str, 'TEAM_ABBREVIATION': str}, index_col=False)\
    
season_box = pd.read_csv('data/box_scores/season_box_scores_20202021.csv',
                         converters={'GAME_ID': str, 'TEAM_ABBREVIATION': str}, index_col=False)
season_games = pd.read_csv('data/games/season_games_20202021.csv', converters={'Game_ID': str, 'TEAM_ABBREVIATION': str}, index_col=False)\

# Define columns to keep from games dataframe
col_keep = ['W_PCT', 'FG_PCT', 'FG3_PCT',
            'FT_PCT', 'OREB', 'AST', 'STL', 'BLK', 'TOV']


In [52]:
# Delete all rows where TEAM_CITY and TEAM_CITY_NAME are not the same
# Error in Data Loading
season_box = season_box[season_box['TEAM_CITY'] == season_box['TEAM_CITY_NAME']]

In [53]:
# Group by GAME_ID in season_20_21_box and keep only first two rows
season_box = season_box.groupby(
    'GAME_ID').head(2).reset_index(drop=True)
# Write to csv
pd.DataFrame.to_csv(season_box, 'tmp.csv', index=False)


In [54]:
# Remove columns not needed
season_box.drop(['LEAGUE_ID', 'TEAM_ID', 'TEAM_CITY', 'GAME_DATE_EST', 'GAME_SEQUENCE',
                 'TEAM_ID.1', 'TEAM_ABBREVIATION.1', 'TEAM_CITY_NAME', 'TEAM_NICKNAME', 'TEAM_WINS_LOSSES', 'TEAM_TURNOVERS', 'TOTAL_TURNOVERS'], axis=1, inplace=True)


In [55]:
season_box.head()
season_box.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1056 entries, 0 to 1055
Data columns (total 25 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   TEAM_ABBREVIATION  1056 non-null   object
 1   PTS_PAINT          1056 non-null   int64 
 2   PTS_2ND_CHANCE     1056 non-null   int64 
 3   PTS_FB             1056 non-null   int64 
 4   LARGEST_LEAD       1056 non-null   int64 
 5   LEAD_CHANGES       1056 non-null   int64 
 6   TIMES_TIED         1056 non-null   int64 
 7   TEAM_REBOUNDS      1056 non-null   int64 
 8   PTS_OFF_TO         1056 non-null   int64 
 9   GAME_ID            1056 non-null   object
 10  PTS_QTR1           1056 non-null   int64 
 11  PTS_QTR2           1056 non-null   int64 
 12  PTS_QTR3           1056 non-null   int64 
 13  PTS_QTR4           1056 non-null   int64 
 14  PTS_OT1            1056 non-null   int64 
 15  PTS_OT2            1056 non-null   int64 
 16  PTS_OT3            1056 non-null   int64 


In [56]:
# Get unique values of Game_ID column in season_20_21_games
game_ids = season_games['Game_ID'].unique()

season_box_ext = pd.DataFrame(columns=season_box.columns)

for id in game_ids:
    # Get all rows with id in Game_ID column
    df_game = season_games[season_games['Game_ID'] == id]
    # Split MATCHUP column string at 'vs.' and strip whitespace
    matchup_split = df_game.iloc[0]['MATCHUP'].split(' ')
    team_one = matchup_split[0].strip()
    team_two = matchup_split[2].strip()
    # Drop columns not in col_keep
    df_game = df_game[col_keep]
    # Make new dataframe out of both rows
    df_team_one = df_game.iloc[0:1]
    df_team_two = df_game.iloc[1:2]
    # # Get row matching TEAM_ABBREVIATION and GAME_ID from season_box
    match_row_one = season_box[(season_box['TEAM_ABBREVIATION'] == team_one) & (
        season_box['GAME_ID'] == id)]
    match_row_two = season_box[(season_box['TEAM_ABBREVIATION'] == team_two) & (
        season_box['GAME_ID'] == id)]
    # Check if match row one is empty
    if match_row_one.empty or match_row_two.empty:
        continue
    # Reset index
    match_row_one.reset_index(drop=True, inplace=True)
    match_row_two.reset_index(drop=True, inplace=True)
    df_team_one.reset_index(drop=True, inplace=True)
    df_team_two.reset_index(drop=True, inplace=True)
    # Concat match_row with df_teams and
    df_match_row_one = pd.concat([match_row_one, df_team_one], axis=1)
    df_match_row_two = pd.concat([match_row_two, df_team_two], axis=1)
    # Concat to season_box_ext
    season_box_ext = pd.concat(
        [season_box_ext, df_match_row_one, df_match_row_two])
    

In [57]:
# write season_box_ext to csv
season_box_ext.to_csv(
    'data/box_scores/season_box_ext_20202021.csv', index=False)
